In [1]:
# Differential expression analysis

library(DESeq2)
library (pheatmap)
library(RColorBrewer)
library (ggplot2)
library(dplyr)

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, cbind, colnames, do.call,
    duplicated, eval, evalq, Filter, Find, get, grep, grepl, intersect,
    is.unsorted, lapply, lengths, Map, mapply, match, mget, order,
    paste, pmax, pmax.int, pmin, pmin.int, Position, rank, rbind,
    Reduce, rownames, sapply, setdiff, sort, table, tapply, union,
    unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The following objects are maske

In [2]:
root="/Users/annasintsova/git_repos/proteus/analysis/DESeq/"

DATA=paste0(root, "data/")
FIG=paste0(root, "figures/")

In [3]:
counts <- read.table("/Users/annasintsova/git_repos/proteus/data/counts/stranded/2018-04-23_counts.csv",
                    row.names =1, sep = ",", header = TRUE)


experiment_info <- read.csv(paste0(root, "../../data/study_design.csv"),
                                header = TRUE, row.names =1)

In [17]:
s_names <- c()
for (c in colnames(counts)){
    x <- gsub("X", "S", unlist(strsplit(c, "_"))[[1]])
    s_names <-c(s_names,x)
}
colnames(counts) <- s_names

In [19]:
experiment_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S63630,L1,Case1,0'',HI,L-HI,NA
S63631,L1,Case1,0'',HI,L-HI,NA
S63632,L1,Case1,0'',HI,L-HI,NA
S63633,L1,Case2,4',HI,L-HI,NA
S63634,L1,Case2,4',HI,L-HI,NA
S63635,L1,Case2,4',HI,L-HI,NA
S63636,L1,Case3,0'',Mix,L-9C,NA
S63638,L1,Case3,0'',Mix,L-9C,NA
S63639,L1,Case4,4',Mix,L-9C,NA
S63641,L1,Case4,4',Mix,L-9C,NA


In [28]:
TimeAnalysisDE <- function (counts, exp_info, filename){
dds <- DESeqDataSetFromMatrix(countData = as.matrix(counts), colData = exp_info,
                                    design = ~time.point)
dds <- DESeq(dds)
res <- results(dds)

resSig <- subset(res, (padj < 0.1)& (log2FoldChange > 1 | log2FoldChange < -1))
write.csv(resSig[order(resSig$log2FoldChange, decreasing = TRUE),], paste0(DATA, filename),
            row.names = TRUE)

output <-  list("results" = resSig[order(resSig$log2FoldChange, decreasing = TRUE),], "dds"= dds)
return (output)
}


In [27]:
exp_info_sample_names <- tibble::rownames_to_column(experiment_info, var = "rownames")


In [26]:
Dienes_line_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "L-9C")[,1]
D_counts <- counts[,Dienes_line_samples]
D_exp_info <- experiment_info[Dienes_line_samples,]
D_exp_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S76079,L2,Case9,0'',Mix,L-9C,7.4
S76080,L2,Case9,0'',Mix,L-9C,7.3
S76081,L2,Case9,0'',Mix,L-9C,6.8
S76082,L2,Case10,30'',Mix,L-9C,6.2
S76083,L2,Case10,30'',Mix,L-9C,6.1
S76084,L2,Case10,30'',Mix,L-9C,5.5


In [30]:
Dienes_time <- TimeAnalysisDE(D_counts, D_exp_info, "2018-04-24-Dienes-line-time.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [33]:
Dienes_time$results

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 36 rows and 6 columns
                         baseMean log2FoldChange     lfcSE      stat
                        <numeric>      <numeric> <numeric> <numeric>
PMI1426                 2603.3788       2.536457 0.2963109  8.560121
PMI1425                 1440.3558       2.219405 0.3207280  6.919898
PMI2396                 2135.7921       2.217305 0.3405866  6.510251
PMI2397                  485.5849       1.802089 0.3317039  5.432823
PMI1427                 2649.0044       1.598964 0.3061397  5.222987
...                           ...            ...       ...       ...
PMI0648                1809.69278      -1.071900 0.2003107 -5.351189
PMI3401                 187.37700      -1.079391 0.2975396 -3.627722
PMI0729                 619.92438      -1.084232 0.3113269 -3.482616
PMI0861               12880.30164      -1.134173 0.2627818 -4.316025
fig|529507.6.peg.3417    40.27845      -1.37

In [34]:
WT_periphery_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "BH")[,1]
WT_periphery_counts <- counts[,WT_periphery_samples]
WT_periphery_exp_info <- experiment_info[WT_periphery_samples,]
WT_periphery_counts

,S76073,S76074,S76075,S76088,S76089,S76090
PMI0001,5071,5627,4484,3439,5636,6074
PMI0002,1547,1875,1452,1331,1935,2153
PMI0003,6233,5985,5254,5338,7894,8399
PMI0004,278,299,237,377,345,381
PMI0005,1462,989,884,628,1044,926
PMI0006,18064,16638,15093,13738,20981,20335
PMI0007,2792,3919,3356,2730,2342,2906
PMI0008,870,1033,931,1279,1216,1308
PMI0009,20533,24041,18494,23161,23117,19901
PMI0010,2381,3621,2653,4067,3135,2886


In [35]:
WT_periphery_time <- TimeAnalysisDE(WT_periphery_counts, WT_periphery_exp_info, "2018-04-24-WT-periphery-line-time.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [37]:
head(WT_periphery_time$results, 25)

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 25 rows and 6 columns
         baseMean log2FoldChange     lfcSE      stat       pvalue         padj
        <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
PMI0348  788.2452       3.448874 0.3217504 10.719097 8.280791e-27 3.043191e-23
PMI0781  412.3377       3.304738 0.3562953  9.275278 1.771556e-20 1.302094e-17
PMI3226 4790.0397       2.899292 0.3588041  8.080432 6.453794e-16 2.156154e-13
PMI1956  167.1570       2.421253 0.3667462  6.601985 4.056877e-11 3.549767e-09
PMI1807  883.3477       2.371982 0.2431928  9.753506 1.782060e-22 1.637267e-19
...           ...            ...       ...       ...          ...          ...
PMI2569  330.6892       1.918191 0.3051307  6.286457 3.247933e-10 2.056655e-08
PMI3595 2161.3947       1.902793 0.3114618  6.109235 1.001095e-09 5.109758e-08
PMI2662 9322.0025       1.901560 0.3474482  5.472931 4.426515e-08 1.196136e-06
PMI

In [38]:
MUT_periphery_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "B9")[,1]
MUT_periphery_counts <- counts[,MUT_periphery_samples]
MUT_periphery_exp_info <- experiment_info[MUT_periphery_samples,]
MUT_periphery_counts

,S76076,S76077,S76078,S76085,S76086,S76087
PMI0001,3397,3732,4374,2375,4001,4212
PMI0002,1065,1353,1808,1058,1496,1624
PMI0003,4504,4394,5639,3552,5134,5832
PMI0004,207,241,326,450,279,324
PMI0005,1524,1209,1019,470,570,611
PMI0006,22358,17849,15837,12023,14745,15306
PMI0007,2588,3079,3872,2556,2259,2252
PMI0008,1024,877,1057,1410,1205,957
PMI0009,20130,29034,17750,24272,28713,24049
PMI0010,2350,3681,3218,4407,4159,3484


In [39]:
MUT_periphery_time <- TimeAnalysisDE(MUT_periphery_counts, MUT_periphery_exp_info, "2018-04-24-MUT-periphery-line-time.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [40]:
MUT_periphery_time$results

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 728 rows and 6 columns
          baseMean log2FoldChange     lfcSE      stat       pvalue         padj
         <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
PMI1031  1118.3504       4.916655 0.3302088 14.889532 3.854608e-50 1.388815e-46
PMI0781   550.5741       4.700586 0.3798781 12.373931 3.616777e-35 6.515624e-32
PMI0348  1505.6800       4.600672 0.5028233  9.149680 5.710249e-20 2.571753e-17
PMI2408   120.5018       3.746797 0.4964789  7.546740 4.462876e-14 5.955459e-12
PMI1956   159.9439       3.744966 0.3915971  9.563313 1.140459e-21 8.218148e-19
...            ...            ...       ...       ...          ...          ...
PMI0913 3785.43253      -3.023720 0.7010666 -4.313029 1.610333e-05 1.534928e-04
PMI3598   50.08999      -3.272011 0.4342601 -7.534681 4.895316e-14 6.299223e-12
PMI0807 2977.62767      -3.285358 0.4683805 -7.014293 2.311142e-12 1.98

In [41]:
Swarm_line_samples <- filter(exp_info_sample_names, lane.ID != "L1" & position == "L-HI")[,1]
Swarm_counts <- counts[,Swarm_line_samples]
Swarm_exp_info <- experiment_info[Swarm_line_samples,]
Swarm_exp_info

,lane.ID,group.ID,time.point,strain.ID,position,RIN
S76067,L2,Case5,30'',HI,L-HI,6.8
S76068,L2,Case5,30'',HI,L-HI,6.1
S76069,L2,Case5,30'',HI,L-HI,6.1
S76070,L2,Case6,0'',HI,L-HI,6.4
S76071,L2,Case6,0'',HI,L-HI,4.5
S76072,L2,Case6,0'',HI,L-HI,6.1


In [44]:
Swarm_time <- TimeAnalysisDE(Swarm_counts, Swarm_exp_info, "2018-04-24-Swarm-line-time.csv")

factor levels were dropped which had no samples
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [45]:
Swarm_time$results

log2 fold change (MAP): time.point 30'' vs 0'' 
Wald test p-value: time.point 30'' vs 0'' 
DataFrame with 121 rows and 6 columns
         baseMean log2FoldChange     lfcSE      stat       pvalue         padj
        <numeric>      <numeric> <numeric> <numeric>    <numeric>    <numeric>
PMI1031  1003.517       2.775656 0.2312591 12.002363 3.452952e-33 1.196448e-29
PMI0437  5635.458       2.063356 0.2094392  9.851812 6.731923e-23 5.831528e-20
PMI3038  1196.089       2.044178 0.2278525  8.971497 2.925126e-19 1.013556e-16
PMI2847  4752.583       1.912011 0.1969639  9.707418 2.803483e-22 1.825954e-19
PMI2254 10780.565       1.862599 0.1613418 11.544432 7.875990e-31 1.364515e-27
...           ...            ...       ...       ...          ...          ...
PMI1426 1939.4264      -1.631109 0.2685475 -6.073820 1.249028e-09 6.658279e-08
PMI0176  219.7845      -1.854880 0.2793990 -6.638824 3.161964e-11 2.235960e-09
PMI1425 1318.4807      -1.966218 0.2666194 -7.374625 1.648088e-13 1.730492e-11
PM